# analysis V4 - new basis code - Dec 6, 2023
- cleaned up the data QC and organizations a bit
- having two anndata and their spin offs - both FISH and proj data 
- flip an axis (y-axis) when plotting - not in data
- focus on L2/3 cells

TODO: 
- separate and organize plotting functions 
- organize the plots and generate more insights

In [ ]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import umap
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib

import anndata 
import scanpy as sc
from scipy.stats import spearmanr

from scroutines import config_plots
from scroutines import basicu
from scroutines import miscu

In [ ]:
import sys
sys.setrecursionlimit(10000)
from scipy.cluster import hierarchy as sch

In [ ]:
def norm_data(adata):
    """This procedure is independnet of each gene
    norm by size (cell volume)
    norm by log2(1+)
    norm by zscore
    """
    # size
    med_size = adata.obs['area'].median()
    max_size = adata.obs['area'].max()
    min_size = adata.obs['area'].min()

    print(f"Min cell size {min_size:.1f} um^3\t  {np.power(min_size,1/3):.1f} um")
    print(f"Med cell size {med_size:.1f} um^3\t  {np.power(med_size,1/3):.1f} um")
    print(f"Max cell size {max_size:.1f} um^3\t  {np.power(max_size,1/3):.1f} um")

    size_factor = (adata.obs['area']/med_size).values


    # norm by size; by log2+1; by zscore
    mat_raw = np.array(adata.X)
    mat_nrm = mat_raw/size_factor.reshape(-1,1) # .divide(size_factor, axis=0)
    mat_log = np.log2(1+mat_nrm)
    mat_zsc = (mat_log - np.mean(mat_log, axis=0))/np.std(mat_log, axis=0)

    adata.layers['nrm'] = mat_nrm
    adata.layers['log'] = mat_log
    adata.layers['zsc'] = mat_zsc
    
    return # mat_raw, mat_nrm, mat_log, mat_zsc

In [ ]:
pth_dat = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt172/proc/r12345v3/'
!ls $pth_dat

In [ ]:
# features
var_names = {
    'r1v3_c0': 'Sorcs3',
    'r1v3_c1': 'Kcnq5',
    'r1v3_c2': 'Chrm2',
    'r1v3_c4': 'Adamts2',
    
    'r2v3_c0': 'Kcnip3',
    'r2v3_c1': 'Rorb',
    'r2v3_c2': 'Cdh13',
    'r2v3_c4': 'Cntn5',
    
    'r3v3_c0': 'Cdh12',
    'r3v3_c1': 'Gria3',
    'r3v3_c2': 'Cntnap2',
    'r3v3_c4': 'Gabrg3',
    
    'r4v3_c0': 'Kcnh5',
    'r4v3_c1': 'RL Cre',
    'r4v3_c2': 'Slc17a7',
    'r4v3_c4': 'Grm8',
    
    'r5v3_c0': 'Ncam2',
    'r5v3_c1': 'Rfx3',
    'r5v3_c2': 'Epha10',
    'r5v3_c4': 'Baz1a',
}
proj_idx = np.array(['r4v3_c1'])

# cells
f_meta = os.path.join(pth_dat, 'roi.csv')

# spots
f_spot = os.path.join(pth_dat, 'spotcount.csv')

In [ ]:
# f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/cheng21_cell_scrna/res/L23-ABC-genes-n288-n286unq-annot_v2.csv'
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/gene_ptime_P28_L23_Mar27.tsv'
df_annot = pd.read_csv(f).sort_values('gene_ptime')

In [ ]:
var = pd.Series(var_names).to_frame('name')
var['proj'] = False
var.loc[proj_idx, 'proj'] = True
var['ptime'] = df_annot.set_index('gene').reindex(var['name'])['gene_ptime'].values
# var['ptime_order'] = var['ptime'].rank()  

var_idx = var.index.values.astype(str)
var_i2n = var['name'] 
var_n2i = var.reset_index().set_index('name')['index']

gene_idx = np.array([idx for idx in var_idx if idx not in proj_idx])

# ftrs_order  = gene_names.index.values 
# genes_order = gene_names.values
# var_oidx = np.argsort(var['ptime'].values)
# var_order = var_i2n[var_idx[var_oidx]]

In [ ]:
var_order = var.sort_values('ptime')['name']

var_order_manual = pd.Series({
    'r2v3_c2': 'Cdh13',
    'r1v3_c4': 'Adamts2',
    'r5v3_c1': 'Rfx3',
    'r2v3_c4': 'Cntn5',
    
    'r3v3_c4': 'Gabrg3',
    'r4v3_c4': 'Grm8',
    
    'r2v3_c0': 'Kcnip3',
    'r5v3_c4': 'Baz1a',
    'r1v3_c0': 'Sorcs3',
    'r1v3_c1': 'Kcnq5',
    'r3v3_c2': 'Cntnap2',
    
    'r3v3_c0': 'Cdh12',
    'r5v3_c2': 'Epha10',
    'r4v3_c0': 'Kcnh5',
    
    'r5v3_c0': 'Ncam2',
    'r1v3_c2': 'Chrm2',
    'r2v3_c1': 'Rorb',
    
    'r3v3_c1': 'Gria3',
    'r4v3_c2': 'Slc17a7',
    'r4v3_c1': 'RL Cre',
    
})


print(var_idx)
print(var_order)
print(var_order_manual)

In [ ]:
# cols
raw_var_idx = np.char.add(var_idx, '_raw')
nrm_var_idx = np.char.add(var_idx, '_nrm')
log_var_idx = np.char.add(var_idx, '_log')
zsc_var_idx = np.char.add(var_idx, '_zsc')

raw_gene_idx = np.char.add(gene_idx, '_raw')
nrm_gene_idx = np.char.add(gene_idx, '_nrm')
log_gene_idx = np.char.add(gene_idx, '_log')
zsc_gene_idx = np.char.add(gene_idx, '_zsc')

In [ ]:
meta = pd.read_csv(f_meta, index_col=0)
spot = pd.read_csv(f_spot, index_col=0)
assert np.all(meta.index.values == spot.index.values)
print(meta.shape, spot.shape) # 

max_x, max_y, max_z = meta[['x', 'y', 'z']].describe().loc['max']
min_x, min_y, min_z = meta[['x', 'y', 'z']].describe().loc['min']
print(f'x: {min_x:.1f}\t{max_x:.1f}')
print(f'y: {min_y:.1f}\t{max_y:.1f}')
print(f'z: {min_z:.1f}\t{max_z:.1f}')

meta['to_edge'] = np.minimum(
    np.minimum(meta['x']-min_x, max_x-meta['x']), 
    np.minimum(meta['y']-min_y, max_y-meta['y']), 
    np.minimum(meta['z']-min_z, max_z-meta['z']),
)
meta['cov'] = spot.sum(axis=1)

# bin data 
bins_8p = np.linspace(0,400,8+1).astype(int)
bins_4p = np.linspace(0,400,4+1).astype(int)
print(bins_8p, bins_4p)

meta['xb_8p'] = pd.cut(meta['x'], bins=bins_8p)
meta['yb_8p'] = pd.cut(meta['y'], bins=bins_8p)
meta['zb_8p'] = pd.cut(meta['z'], bins=bins_8p)

meta['xb_4p'] = pd.cut(meta['x'], bins=bins_4p)
meta['yb_4p'] = pd.cut(meta['y'], bins=bins_4p)
meta['zb_4p'] = pd.cut(meta['z'], bins=bins_4p)

In [ ]:
adata = anndata.AnnData(X=spot.values, obs=meta, var=var)
adata

In [ ]:
# remove outliers adata -> adata2
df = adata.obs
conds = [
    df['area'] > 500,
    df['x'] > min_x + 20,
    df['x'] < max_x - 20,
    
    df['y'] > min_y + 20,
    df['y'] < max_y - 20,
    
    df['z'] > min_z + 20,
    df['z'] < max_z - 20,
]
cond_all = np.ones(len(df)) > 0
for cond in conds:
    cond_all = np.logical_and(cond_all, cond)
    print(cond_all.sum())
print(f"Num cells before and after: {len(df)} -> {cond_all.sum()}")

# remove outliers
adata2 = adata[cond_all].copy()
adata2

In [ ]:
# separate the proj data adata2 -> adata3
adata3 = adata2[:,gene_idx].copy()
adata3.obs[proj_idx] = np.array(adata2[:,proj_idx].X)
adata3.obs['cov_gene'] = np.array(adata3.X.sum(axis=1))

In [ ]:
# normalize the data and record
norm_data(adata2)
df_p2 = adata2.obs.copy()
df_p2[nrm_var_idx] = np.array(adata2.layers['nrm'])

norm_data(adata3)
df_p3 = adata3.obs.copy()
df_p3[nrm_gene_idx] = np.array(adata3.layers['nrm'])

# report 

In [ ]:
for idx in var_idx:
    val = adata[:,idx].X[:,0]
    print(f'{idx}\t{var_i2n.loc[idx]}\t{100*np.sum(val>0)/len(val):.2f}%\t{np.min(val):.1f}\t{np.median(val):.1f}\t{np.percentile(val, 99):.1f}\t{np.max(val):.1f}')

In [ ]:
cols = ['x', 'y', 'z', 'area']
with sns.plotting_context('paper'):
    fig, axs = plt.subplots(4, 1, figsize=(1*6, 2*4))
    for ax, col in zip(axs, cols):
        sns.histplot(adata.obs[col], ax=ax)
        ax.set_xlabel(col)
    fig.subplots_adjust(hspace=0.5)
    plt.show()


In [ ]:
sns.scatterplot(data=adata3.obs, x='area', y='cov', s=2, edgecolor='none')
sns.scatterplot(data=adata3.obs, x='area', y='cov_gene', s=2, edgecolor='none')
plt.show()

plt.scatter(np.log2(adata3.obs['area']), 
            np.log2(adata3.obs['cov']),
            s=2, edgecolor='none',
           )
plt.scatter(np.log2(adata3.obs['area']), 
            np.log2(adata3.obs['cov_gene']),
            s=2, edgecolor='none',
           )

In [ ]:
with sns.plotting_context('paper'):
    fig, axs = plt.subplots(2, 1, figsize=(10*1,4*2))
    ax = axs[0]
    sns.boxplot(data=adata.X, ax=ax)
    ax.set_xticklabels(adata.var.index.values, rotation=90) 
    ax.set_ylabel('counts')
    ax.set_xlabel('Genes')
    sns.despine(ax=ax)

    ax = axs[1]
    sns.boxplot(data=adata.X, ax=ax)
    ax.set_xticklabels(adata.var['name'].values, rotation=90)
    sns.despine(ax=ax)
    ax.set_ylim([0,50])
    ax.set_ylabel('counts')
    ax.set_xlabel('Genes')

# z-sectioning visuals
- bin into zbin
- plot for each zbin

In [ ]:
sys.path.insert(0, '../')
import plotting_easifish

import importlib
importlib.reload(plotting_easifish)

from plotting_easifish import view_z_sections
from plotting_easifish import view_z_sections_4panels
from plotting_easifish import view_z_sections_labels
from plotting_easifish import gen_discrete_colors


In [ ]:

# umap, neighbors
ucs = umap.UMAP(n_components=2, n_neighbors=15, random_state=0).fit_transform(adata3.layers['zsc'])
print(ucs.shape)
adata2.obs['ux'] = ucs[:,0]
adata2.obs['uy'] = ucs[:,1]
adata3.obs['ux'] = ucs[:,0]
adata3.obs['uy'] = ucs[:,1]

df_p2['ux'] = ucs[:,0]
df_p2['uy'] = ucs[:,1]
df_p3['ux'] = ucs[:,0]
df_p3['uy'] = ucs[:,1]

# neighbors, leiden
adata3.obsm['zsc'] = adata3.layers['zsc']
sc.pp.neighbors(adata3, n_neighbors=15, use_rep='zsc')
palettes = {}
dismaps = {}

for i, r in enumerate([0.5, 1, 2]):
    key = f'clst_l{i+1}'
    sc.tl.leiden(adata3, resolution=r, key_added=key)
    clsts = 1+adata3.obs[key].astype(int) # .unique()
    palette, dismap = gen_discrete_colors(len(np.unique(clsts)))
    
    depth = adata3.obs['y']
    depth_rank = pd.DataFrame(np.array([depth, clsts]).T, columns=['depth', 'clsts']).groupby('clsts').mean().rank(method='dense').astype(int)
    clsts = depth_rank.loc[clsts].values
    
    # register
    palettes[key] = palette
    dismaps[key] = dismap
    adata3.obs[key] = clsts
    adata2.obs[key] = clsts
    df_p3[key] = clsts
    df_p2[key] = clsts
    


In [ ]:
sp_x, sp_y = 'x', 'y'
fig, axs = plt.subplots(5,4,figsize=(4*4,4*5), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_order_manual.index.values, axs.flat)):
    x = df_p2[sp_x].values
    y = df_p2[sp_y].values
    c = df_p2[col+'_nrm'].values
    vmax=np.percentile(c, 95)
    vmin=np.percentile(c,  5)
    c = (c-vmin)/(vmax-vmin)
    
    g = ax.scatter(x, y, c=c, s=5, edgecolor='none', cmap='gray_r', vmax=1, vmin=0) #vmax, vmin=-0.1*vmax)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.grid(False)
    ax.axis('off')
    
    ax.hlines([0,100,200,300], xmin=0, xmax=20, linestyle='--', color='black', zorder=0, linewidth=1)
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(5-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'ux', 'uy'
fig, axs = plt.subplots(5,4,figsize=(4*4,4*5), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_order_manual.index.values, axs.flat)):
    x = df_p2[sp_x].values
    y = df_p2[sp_y].values
    c = df_p2[col+'_nrm'].values
    vmax=np.percentile(c, 95)
    vmin=np.percentile(c,  5)
    c = (c-vmin)/(vmax-vmin)
    
    ax.axis('off')
    ax.grid(False)
    g = ax.scatter(x, y, c=c, s=3, edgecolor='none', cmap='viridis', vmax=1, vmin=0)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(5-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
for sp_x, sp_y in [
    ['x', 'y'],
    ['ux', 'uy'],
    ]:
    fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
    axs.flat[0].invert_yaxis()
    for i, (col, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], axs.flat)):
        palette = palettes[col]
        dismap = dismaps[col]
        x = adata3.obs[sp_x].values
        y = adata3.obs[sp_y].values
        c = adata3.obs[col].values

        ax.grid(False)
        g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
        sns.despine(ax=ax)
        ax.set_title(col)
        ax.set_xlabel(sp_x)
        ax.set_ylabel(sp_y)
        ax.set_aspect('equal')
        ax.axis('off')
        fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)

    fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
col = 'clst_l2'
clst_order_manual = None
# clst_order_manual = [5,2,3,1,6,4,7]

for sp_x, sp_y in [
    ['x', 'y'],
    ['ux', 'uy'],
    ]:
    palette = palettes[col]
    dismap = dismaps[col]
    n = len(adata3.obs[col].unique())
    nx = min(6,n)
    ny = int((n+nx-1)/nx)

    if clst_order_manual is not None:
        clst_order = clst_order_manual
    else:
        clst_order = np.arange(n)+1

    fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
    axs.flat[0].invert_yaxis()
    for i, ax in zip(clst_order, axs.flat):
        x = adata3.obs[sp_x].values
        y = adata3.obs[sp_y].values
        c = adata3.obs[col].values

        ax.grid(False)
        g = ax.scatter(x, y, c=[palette[_c] if i==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)

        ax.set_title(f"C{i}")
        ax.set_xlabel(sp_x)
        ax.set_ylabel(sp_y)
        ax.set_aspect('equal')
        ax.axis('off')

    for ax in axs.flat[n-1:]:
        ax.axis('off')

    fig.subplots_adjust(hspace=0.1, wspace=0.02)
    
# heatmap
dfmean = df_p2.groupby(col)[np.char.add(var_order_manual.index.values.astype(str), '_nrm')].mean()
dfmean = basicu.zscore(dfmean, axis=0)
if clst_order_manual is not None:
    dfmean = dfmean.reindex(clst_order_manual)
yticklabels = var_order_manual.values

fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=yticklabels, 
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))

# round 2 - keep only L2/3/4 neurons 
- include Rorb+ but not Cntn5+/Gria3+/Kcnip3+/Chrm2-

In [ ]:
# remove_these = [4,10,11, ]
remove_these = [6,7,11,12] # keep peak Rorb population
adata2v2 = adata2[~adata2.obs['clst_l2'].isin(remove_these)].copy()
adata3v2 = adata3[~adata3.obs['clst_l2'].isin(remove_these)].copy()
print(adata2v2.shape, adata3v2.shape)

In [ ]:
df_p2v2 = adata2v2.obs.copy()
df_p2v2[nrm_var_idx] = np.array(adata2v2.layers['nrm'])
df_p3v2 = adata3v2.obs.copy()
df_p3v2[nrm_gene_idx] = np.array(adata3v2.layers['nrm'])

In [ ]:
# # normalize the data and record
# norm_data(adata2v2)
# norm_data(adata3v2)

In [ ]:


# umap, neighbors
ucs = umap.UMAP(n_components=2, n_neighbors=15, random_state=0).fit_transform(adata3v2.layers['zsc'])
print(ucs.shape)
adata2v2.obs['ux'] = ucs[:,0]
adata2v2.obs['uy'] = ucs[:,1]
adata3v2.obs['ux'] = ucs[:,0]
adata3v2.obs['uy'] = ucs[:,1]

df_p2v2['ux'] = ucs[:,0]
df_p2v2['uy'] = ucs[:,1]
df_p3v2['ux'] = ucs[:,0]
df_p3v2['uy'] = ucs[:,1]

# neighbors, leiden
adata3v2.obsm['zsc'] = adata3v2.layers['zsc']
sc.pp.neighbors(adata3v2, n_neighbors=15, use_rep='zsc')
palettes = {}
dismaps = {}

for i, r in enumerate([0.5, 1, 2]):
    key = f'clst_l{i+1}'
    sc.tl.leiden(adata3v2, resolution=r, key_added=key)
    clsts = 1+adata3v2.obs[key].astype(int) # .unique()
    palette, dismap = gen_discrete_colors(len(np.unique(clsts)))
    
    depth = adata3v2.obs['y']
    depth_rank = pd.DataFrame(np.array([depth, clsts]).T, columns=['depth', 'clsts']).groupby('clsts').mean().rank(method='dense').astype(int)
    clsts = depth_rank.loc[clsts].values
    
    # register
    palettes[key] = palette
    dismaps[key] = dismap
    adata3v2.obs[key] = clsts
    adata2v2.obs[key] = clsts
    df_p3v2[key] = clsts
    df_p2v2[key] = clsts

In [ ]:
for sp_x, sp_y in [
    ['x', 'y'],
    ['ux', 'uy'],
    ]:
    fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
    axs.flat[0].invert_yaxis()
    for i, (col, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], axs.flat)):
        palette = palettes[col]
        dismap = dismaps[col]
        x = adata3v2.obs[sp_x].values
        y = adata3v2.obs[sp_y].values
        c = adata3v2.obs[col].values

        ax.grid(False)
        g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
        sns.despine(ax=ax)
        ax.set_title(col)
        ax.set_xlabel(sp_x)
        ax.set_ylabel(sp_y)
        ax.set_aspect('equal')
        ax.axis('off')
        fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)

    fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
col = 'clst_l2'
clst_order_manual = None
# clst_order_manual = [5,2,3,1,6,4,7]

for sp_x, sp_y in [
    ['x', 'y'],
    ['ux', 'uy'],
    ]:
    palette = palettes[col]
    dismap = dismaps[col]
    n = len(adata3.obs[col].unique())
    nx = min(6,n)
    ny = int((n+nx-1)/nx)

    if clst_order_manual is not None:
        clst_order = clst_order_manual
    else:
        clst_order = np.arange(n)+1

    fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
    axs.flat[0].invert_yaxis()
    for i, ax in zip(clst_order, axs.flat):
        x = adata3v2.obs[sp_x].values
        y = adata3v2.obs[sp_y].values
        c = adata3v2.obs[col].values

        ax.grid(False)
        g = ax.scatter(x, y, c=[palette[_c] if i==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)

        ax.set_title(f"C{i}")
        ax.set_xlabel(sp_x)
        ax.set_ylabel(sp_y)
        ax.set_aspect('equal')
        ax.axis('off')

    for ax in axs.flat[n-1:]:
        ax.axis('off')

    fig.subplots_adjust(hspace=0.1, wspace=0.02)
    
# heatmap
dfmean = df_p2v2.groupby(col)[np.char.add(var_order_manual.index.values.astype(str), '_nrm')].mean()
dfmean = basicu.zscore(dfmean, axis=0)
if clst_order_manual is not None:
    dfmean = dfmean.reindex(clst_order_manual)
yticklabels = var_order_manual.values

fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=yticklabels, 
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))

In [ ]:
sp_x, sp_y = 'x', 'y'
fig, axs = plt.subplots(5,4,figsize=(4*4,4*5), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_order_manual.index.values, axs.flat)):
    x = df_p2v2[sp_x].values
    y = df_p2v2[sp_y].values
    c = df_p2v2[col+'_nrm'].values
    vmax=np.percentile(c, 95)
    vmin=np.percentile(c,  5)
    c = (c-vmin)/(vmax-vmin)
    
    g = ax.scatter(x, y, c=c, s=5, edgecolor='none', cmap='gray_r', vmax=1, vmin=0) #vmax, vmin=-0.1*vmax)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.grid(False)
    ax.axis('off')
    
    ax.hlines([0,100,200,300], xmin=0, xmax=20, linestyle='--', color='black', zorder=0, linewidth=1)
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(5-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'ux', 'uy'
fig, axs = plt.subplots(5,4,figsize=(4*4,4*5), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_order_manual.index.values, axs.flat)):
    x = df_p2v2[sp_x].values
    y = df_p2v2[sp_y].values
    c = df_p2v2[col+'_nrm'].values
    vmax=np.percentile(c, 95)
    vmin=np.percentile(c,  5)
    c = (c-vmin)/(vmax-vmin)
    
    ax.axis('off')
    ax.grid(False)
    g = ax.scatter(x, y, c=c, s=3, edgecolor='none', cmap='viridis', vmax=1, vmin=0)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(5-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

# order by the continuum

In [ ]:
mat_ftrs = pd.DataFrame(adata3v2.layers['zsc'], index=adata3v2.obs.index, columns=adata3v2.var.index)
mat_prjs = adata3v2.obs[proj_idx] 

row_order = np.argsort(adata3v2.obs['y']).values # .sort_values().index
col_order = basicu.get_index_from_array(mat_ftrs.columns, var_order_manual.index.values[:-1])
col_order

In [ ]:
pca = PCA(n_components=10)
pcs = pca.fit_transform(mat_ftrs.values)

adata3v2.obsm['X_pca'] = pcs 
sc.pp.neighbors(adata3v2, n_neighbors=30, use_rep='X_pca')
sc.tl.diffmap(adata3v2)
adata3v2.uns['iroot'] = np.argmin(-pcs[:,0])
sc.tl.dpt(adata3v2)

df_p2v2['pc1'] = pcs[:,0]
df_p2v2['pc2'] = pcs[:,1]
df_p2v2['pc3'] = pcs[:,2]
df_p2v2['pc4'] = pcs[:,3]
df_p2v2['ptime'] = adata3v2.obs['dpt_pseudotime'].values

plt.plot(pca.explained_variance_ratio_, '-o')
pcs.shape

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*5,1*4), sharex=True, sharey=True)
ax = axs[0]
palette2 = palettes['clst_l2']
sns.scatterplot(data=df_p2v2, x='pc1', y='pc2', hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.set_title('cluster')
ax.grid(False)
ax.set_aspect('equal')
sns.despine(ax=ax)

ax = axs[1]
p = ax.scatter(df_p2v2['pc1'], df_p2v2['pc2'], c=df_p2v2['ptime'], s=5, edgecolor='none', cmap='rocket_r')
fig.colorbar(p, shrink=0.5)
ax.set_title('ptime')
ax.grid(False)
ax.set_aspect('equal')
sns.despine(ax=ax)

ax = axs[2]
p = ax.scatter(df_p2v2['pc1'], df_p2v2['pc2'], c=df_p2v2['y'], s=5, edgecolor='none', cmap='rocket_r')
fig.colorbar(p, shrink=0.5)
ax.set_title('y')
ax.grid(False)
ax.set_aspect('equal')
sns.despine(ax=ax)

# fig.tight_layout()
plt.show()


In [ ]:
y = 'ptime'
xs = ['y', 'x', 'z']
fig, axs = plt.subplots(1,3,figsize=(3*5,1*4), sharey=True)
for i, x in enumerate(xs):
    ax = axs[i]
    a = df_p2v2[x] 
    b = df_p2v2[y] 
    r, p = spearmanr(a, b)
    if i == 2:
        legend = True
    else:
        legend = False
    sns.scatterplot(data=df_p2v2, x=x, y=y, hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax, legend=legend)
    ax.set_title(f'Spearman r={r:.2f}, p={p:.1e}', fontsize=10)
    ax.grid(False)
    sns.despine(ax=ax)
ax.legend(bbox_to_anchor=(1,1))

ax.invert_yaxis()
plt.show()

In [ ]:
y = 'y'
xs = ['y', 'x', 'z']
fig, axs = plt.subplots(1,3,figsize=(3*5,1*4), sharey=True)
for i, x in enumerate(xs):
    ax = axs[i]
    a = df_p2v2[x] 
    b = df_p2v2[y] 
    r, p = spearmanr(a, b)
    if i == 2:
        legend = True
    else:
        legend = False
    sns.scatterplot(data=df_p2v2, x=x, y=y, hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax, legend=legend)
    ax.set_title(f'Spearman r={r:.2f}, p={p:.1e}', fontsize=10)
    ax.grid(False)
    sns.despine(ax=ax)
ax.legend(bbox_to_anchor=(1,1))

ax.invert_yaxis()
plt.show()

In [ ]:
x = 'y'
y = 'ptime'
fig, axs = plt.subplots(4,5,figsize=(5*4,4*4), sharex=True, sharey=True)
for col, ax in zip(var_order_manual.index, axs.flat):
    hue = col+'_nrm'
    c = df_p2v2[hue].values
    vmax = np.percentile(c,95) 
    vmin = np.percentile(c,5)
    # vmin = -0.1*vmax # np.percentile(c,0) 
    g = sns.scatterplot(data=df_p2v2, x=x, y=y, c=c, cmap='viridis',# 'gray_r', # hue=hue, 
                    vmin=vmin, vmax=vmax, s=3, edgecolor='none', ax=ax)
    ax.set_title(col+'_'+var_i2n[col])
    ax.grid(False)
    sns.despine(ax=ax)
    ax.set_xlim(xmin=0, xmax=400)
    ax.set_xticks([0,100,200,300,400])
ax.invert_yaxis()
plt.show()

In [ ]:
x = 'y'
_x = df_p2v2[x].values
df_p2v2['bin_nrm'], bins = pd.qcut(_x, 20, retbins=True)
df_p2v2_mean = df_p2v2.filter(regex="nrm$", axis=1).groupby('bin_nrm').mean(numeric_only=True)

bins_mid = np.mean(np.vstack([bins[:-1], bins[1:]]), axis=0)

clst = 'clst_l2'
palette = palettes[clst]

fig, axs = plt.subplots(4,5,figsize=(5*4,4*4), sharex=True) #, sharey=True)
for col, ax in zip(var_order_manual.index, axs.flat):
    hue = col+'_nrm'
    _y = df_p2v2[hue].values
    _y2 = df_p2v2_mean[hue].values
    
    c = [palette[lbl] for lbl in df_p2v2[clst]]
    
    vmax = np.percentile(_y,95) 
    vmin = np.percentile(_y,5)
    
    g = ax.scatter(_x, _y, c=c, s=3, edgecolor='none')
    g2 = ax.plot(bins_mid, _y2, '-o', color='k', markersize=5)
    
    ax.set_title(col+'_'+var_i2n[col])
    ax.set_ylim([vmin, vmax])
    ax.grid(False)
    sns.despine(ax=ax)
plt.show()

In [ ]:
x = 'ptime'
_x = df_p2v2[x].values
df_p2v2['bin_nrm'], bins = pd.qcut(_x, 20, retbins=True)
df_p2v2_mean = df_p2v2.filter(regex="nrm$", axis=1).groupby('bin_nrm').mean(numeric_only=True)

bins_mid = np.mean(np.vstack([bins[:-1], bins[1:]]), axis=0)

clst = 'clst_l2'
palette = palettes[clst]

fig, axs = plt.subplots(4,5,figsize=(5*4,4*4), sharex=True) #, sharey=True)
for col, ax in zip(var_order_manual.index, axs.flat):
    hue = col+'_nrm'
    _y = df_p2v2[hue].values
    _y2 = df_p2v2_mean[hue].values
    
    vmax = np.percentile(_y,95) 
    vmin = np.percentile(_y,5)
    
    c = [palette[lbl] for lbl in df_p2v2[clst]]
    
    g = ax.scatter(_x, _y, c=c, s=3, edgecolor='none')
    g2 = ax.plot(bins_mid, _y2, '-o', color='k', markersize=5)
    
    ax.set_title(col+'_'+var_i2n[col])
    ax.set_ylim([vmin, vmax])
    ax.grid(False)
    sns.despine(ax=ax)
    # ax.set_xlim(xmin=0, xmax=400)
    # ax.set_xticks([0,100,200,300,400])
# ax.invert_yaxis()
plt.show()

In [ ]:
row_order = np.argsort(df_p2v2['ptime'].values)
col_order = basicu.get_index_from_array(mat_ftrs.columns, var_order_manual.index.values[:-1])

fig = plt.figure(figsize=(20,8))
axd = fig.subplot_mosaic("A\n"*12+"B\nC\nD\nE\nF")
ax = axd["A"]
col_idx = mat_ftrs.columns[col_order].values.astype(str)
col_names = var_i2n[col_idx].values.astype(str)
yticklabels = np.char.add(np.char.add(col_names, ' '), col_idx)

sns.heatmap(
    mat_ftrs.iloc[row_order,col_order].T,
    xticklabels=False,
    yticklabels=yticklabels,
    cmap='coolwarm',
    cbar_kws=dict(shrink=0.5, label='zscore log2(1+norm expr.)'),
    vmax=3, vmin=-3,
    ax=ax,
)
ax.set_ylabel('Cells')
# ax.set_xticklabels()

for i, (ax, col) in enumerate(zip(
    (axd["B"], axd["C"], axd["D"]),
    ('clst_l1', 'clst_l2', 'clst_l3'),
    )):
    # vec = adata3v2.obs[col][row_order].values
    sns.heatmap(
        adata3v2.obs[[col]].rename(columns={col: col.replace('clst_l', 'L')}).iloc[row_order].T,
        xticklabels=False,
        cmap='jet',
        cbar_kws=dict(aspect=5, label='', ticks=[]),
        ax=ax,
    )
    ax.set_ylabel('')
    # positions = np.cumsum(np.unique(vec, return_counts=True)[1])
    # ax.hlines(positions, xmin=-0.2, xmax=1, linewidth=1, linestyle='-', color='k', clip_on=False)
    # for i, pos in enumerate(positions):
    #     ax.text(0, pos, i+1, color='white', fontsize=15)

ax = axd["E"]
g = sns.heatmap(
    mat_prjs.iloc[row_order].T,
    xticklabels=False,
    cmap='Greys',
    cbar_kws=dict(aspect=5, ),
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points

ax = axd["F"]
g = sns.heatmap(
    adata3v2.obs[['y']].iloc[row_order].T, 
    xticklabels=False,
    cmap='Greys',
    cbar_kws=dict(aspect=5, label='um'),
    # vmin=0,
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points

# round 3 - keep only L2/3 ABC neurons 
- Gria3 / Rorb / sharp pseudotime / ...

In [ ]:
# remove_these = [4,10,11, ]
remove_these = [6,7,8] # keep peak Rorb population
adata2v3 = adata2v2[~adata2v2.obs['clst_l2'].isin(remove_these)].copy()
adata3v3 = adata3v2[~adata3v2.obs['clst_l2'].isin(remove_these)].copy()
print(adata2v3.shape, adata3v3.shape)

In [ ]:
df_p2v3 = adata2v3.obs.copy()
df_p2v3[nrm_var_idx] = np.array(adata2v3.layers['nrm'])
df_p3v3 = adata3v3.obs.copy()
df_p3v3[nrm_gene_idx] = np.array(adata3v3.layers['nrm'])

In [ ]:
# normalize the data and record
norm_data(adata2v3)
norm_data(adata3v3)

In [ ]:
# umap, neighbors
ucs = umap.UMAP(n_components=2, n_neighbors=15, random_state=0).fit_transform(adata3v3.layers['zsc'])
print(ucs.shape)
adata2v3.obs['ux'] = ucs[:,0]
adata2v3.obs['uy'] = ucs[:,1]
adata3v3.obs['ux'] = ucs[:,0]
adata3v3.obs['uy'] = ucs[:,1]

df_p2v3['ux'] = ucs[:,0]
df_p2v3['uy'] = ucs[:,1]
df_p3v3['ux'] = ucs[:,0]
df_p3v3['uy'] = ucs[:,1]

# neighbors, leiden
adata3v3.obsm['zsc'] = adata3v3.layers['zsc']
sc.pp.neighbors(adata3v3, n_neighbors=15, use_rep='zsc')
palettes = {}
dismaps = {}

for i, r in enumerate([0.5, 1, 2]):
    key = f'clst_l{i+1}'
    sc.tl.leiden(adata3v3, resolution=r, key_added=key)
    clsts = 1+adata3v3.obs[key].astype(int) # .unique()
    palette, dismap = gen_discrete_colors(len(np.unique(clsts)))
    
    depth = adata3v3.obs['y']
    depth_rank = pd.DataFrame(np.array([depth, clsts]).T, columns=['depth', 'clsts']).groupby('clsts').mean().rank(method='dense').astype(int)
    clsts = depth_rank.loc[clsts].values
    
    # register
    palettes[key] = palette
    dismaps[key] = dismap
    adata3v3.obs[key] = clsts
    adata2v3.obs[key] = clsts
    df_p3v3[key] = clsts
    df_p2v3[key] = clsts

In [ ]:
for sp_x, sp_y in [
    ['x', 'y'],
    ['ux', 'uy'],
    ]:
    fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
    axs.flat[0].invert_yaxis()
    for i, (col, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], axs.flat)):
        palette = palettes[col]
        dismap = dismaps[col]
        x = adata3v3.obs[sp_x].values
        y = adata3v3.obs[sp_y].values
        c = adata3v3.obs[col].values

        ax.grid(False)
        g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
        sns.despine(ax=ax)
        ax.set_title(col)
        ax.set_xlabel(sp_x)
        ax.set_ylabel(sp_y)
        ax.set_aspect('equal')
        ax.axis('off')
        fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)

    fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
col = 'clst_l2'
clst_order_manual = None
# clst_order_manual = [5,2,3,1,6,4,7]

for sp_x, sp_y in [
    ['x', 'y'],
    ['ux', 'uy'],
    ]:
    palette = palettes[col]
    dismap = dismaps[col]
    n = len(adata3v3.obs[col].unique())
    nx = min(6,n)
    ny = int((n+nx-1)/nx)

    if clst_order_manual is not None:
        clst_order = clst_order_manual
    else:
        clst_order = np.arange(n)+1

    fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
    axs.flat[0].invert_yaxis()
    for i, ax in zip(clst_order, axs.flat):
        x = adata3v3.obs[sp_x].values
        y = adata3v3.obs[sp_y].values
        c = adata3v3.obs[col].values

        ax.grid(False)
        g = ax.scatter(x, y, c=[palette[_c] if i==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)

        ax.set_title(f"C{i}")
        ax.set_xlabel(sp_x)
        ax.set_ylabel(sp_y)
        ax.set_aspect('equal')
        ax.axis('off')

    for ax in axs.flat[n-1:]:
        ax.axis('off')

    fig.subplots_adjust(hspace=0.1, wspace=0.02)
    
# heatmap
dfmean = df_p2v3.groupby(col)[np.char.add(var_order_manual.index.values.astype(str), '_nrm')].mean()
dfmean = basicu.zscore(dfmean, axis=0)
if clst_order_manual is not None:
    dfmean = dfmean.reindex(clst_order_manual)
yticklabels = var_order_manual.values

fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=yticklabels, 
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))

In [ ]:
sp_x, sp_y = 'x', 'y'
fig, axs = plt.subplots(5,4,figsize=(4*4,4*5), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_order_manual.index.values, axs.flat)):
    x = df_p2v3[sp_x].values
    y = df_p2v3[sp_y].values
    c = df_p2v3[col+'_nrm'].values
    vmax=np.percentile(c, 95)
    vmin=np.percentile(c,  5)
    c = (c-vmin)/(vmax-vmin)
    
    g = ax.scatter(x, y, c=c, s=5, edgecolor='none', cmap='gray_r', vmax=1, vmin=0) #vmax, vmin=-0.1*vmax)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.grid(False)
    ax.axis('off')
    
    ax.hlines([0,100,200,300], xmin=0, xmax=20, linestyle='--', color='black', zorder=0, linewidth=1)
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(5-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'ux', 'uy'
fig, axs = plt.subplots(5,4,figsize=(4*4,4*5), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_order_manual.index.values, axs.flat)):
    x = df_p2v3[sp_x].values
    y = df_p2v3[sp_y].values
    c = df_p2v3[col+'_nrm'].values
    vmax=np.percentile(c, 95)
    vmin=np.percentile(c,  5)
    c = (c-vmin)/(vmax-vmin)
    
    ax.axis('off')
    ax.grid(False)
    g = ax.scatter(x, y, c=c, s=3, edgecolor='none', cmap='viridis', vmax=1, vmin=0)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(5-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

# order by the continuum

In [ ]:
mat_ftrs = pd.DataFrame(adata3v3.layers['zsc'], index=adata3v3.obs.index, columns=adata3v3.var.index)
mat_prjs = adata3v3.obs[proj_idx] 

row_order = np.argsort(adata3v3.obs['y']).values # .sort_values().index
col_order = basicu.get_index_from_array(mat_ftrs.columns, var_order_manual.index.values[:-1])
col_order

In [ ]:
pca = PCA(n_components=10)
pcs = pca.fit_transform(mat_ftrs.values)

adata3v3.obsm['X_pca'] = pcs 
sc.pp.neighbors(adata3v2, n_neighbors=30, use_rep='X_pca')
sc.tl.diffmap(adata3v2)
adata3v3.uns['iroot'] = np.argmin(-pcs[:,0])
sc.tl.dpt(adata3v2)

df_p2v3['pc1'] = pcs[:,0]
df_p2v3['pc2'] = pcs[:,1]
df_p2v3['pc3'] = pcs[:,2]
df_p2v3['pc4'] = pcs[:,3]
df_p2v3['ptime'] = adata3v3.obs['dpt_pseudotime'].values

plt.plot(pca.explained_variance_ratio_, '-o')
pcs.shape

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*5,1*4), sharex=True, sharey=True)
ax = axs[0]
palette2 = palettes['clst_l2']
sns.scatterplot(data=df_p2v3, x='pc1', y='pc2', hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.set_title('cluster')
ax.grid(False)
ax.set_aspect('equal')
sns.despine(ax=ax)

ax = axs[1]
p = ax.scatter(df_p2v3['pc1'], df_p2v3['pc2'], c=df_p2v3['ptime'], s=5, edgecolor='none', cmap='rocket_r')
fig.colorbar(p, shrink=0.5)
ax.set_title('ptime')
ax.grid(False)
ax.set_aspect('equal')
sns.despine(ax=ax)

ax = axs[2]
p = ax.scatter(df_p2v3['pc1'], df_p2v3['pc2'], c=df_p2v3['y'], s=5, edgecolor='none', cmap='rocket_r')
fig.colorbar(p, shrink=0.5)
ax.set_title('y')
ax.grid(False)
ax.set_aspect('equal')
sns.despine(ax=ax)

# fig.tight_layout()
plt.show()


In [ ]:
y = 'ptime'
xs = ['y', 'x', 'z']
fig, axs = plt.subplots(1,3,figsize=(3*5,1*4), sharey=True)
for i, x in enumerate(xs):
    ax = axs[i]
    a = df_p2v3[x] 
    b = df_p2v3[y] 
    r, p = spearmanr(a, b)
    if i == 2:
        legend = True
    else:
        legend = False
    sns.scatterplot(data=df_p2v3, x=x, y=y, hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax, legend=legend)
    ax.set_title(f'Spearman r={r:.2f}, p={p:.1e}', fontsize=10)
    ax.grid(False)
    sns.despine(ax=ax)
ax.legend(bbox_to_anchor=(1,1))

ax.invert_yaxis()
plt.show()

In [ ]:
y = 'y'
xs = ['y', 'x', 'z']
fig, axs = plt.subplots(1,3,figsize=(3*5,1*4), sharey=True)
for i, x in enumerate(xs):
    ax = axs[i]
    a = df_p2v3[x] 
    b = df_p2v3[y] 
    r, p = spearmanr(a, b)
    if i == 2:
        legend = True
    else:
        legend = False
    sns.scatterplot(data=df_p2v3, x=x, y=y, hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax, legend=legend)
    ax.set_title(f'Spearman r={r:.2f}, p={p:.1e}', fontsize=10)
    ax.grid(False)
    sns.despine(ax=ax)
ax.legend(bbox_to_anchor=(1,1))

ax.invert_yaxis()
plt.show()

In [ ]:
x = 'y'
y = 'ptime'
fig, axs = plt.subplots(4,5,figsize=(5*4,4*4), sharex=True, sharey=True)
for col, ax in zip(var_order_manual.index, axs.flat):
    hue = col+'_nrm'
    c = df_p2v3[hue].values
    vmax = np.percentile(c,95) 
    vmin = np.percentile(c,5)
    # vmin = -0.1*vmax # np.percentile(c,0) 
    g = sns.scatterplot(data=df_p2v3, x=x, y=y, c=c, cmap='viridis',# 'gray_r', # hue=hue, 
                    vmin=vmin, vmax=vmax, s=3, edgecolor='none', ax=ax)
    ax.set_title(col+'_'+var_i2n[col])
    ax.grid(False)
    sns.despine(ax=ax)
    ax.set_xlim(xmin=0, xmax=400)
    ax.set_xticks([0,100,200,300,400])
ax.invert_yaxis()
plt.show()

In [ ]:
x = 'y'
_x = df_p2v3[x].values
df_p2v3['bin_nrm'], bins = pd.qcut(_x, 20, retbins=True)
df_p2v3_mean = df_p2v3.filter(regex="nrm$", axis=1).groupby('bin_nrm').mean(numeric_only=True)

bins_mid = np.mean(np.vstack([bins[:-1], bins[1:]]), axis=0)

clst = 'clst_l2'
palette = palettes[clst]

fig, axs = plt.subplots(4,5,figsize=(5*4,4*4), sharex=True) #, sharey=True)
for col, ax in zip(var_order_manual.index, axs.flat):
    hue = col+'_nrm'
    _y = df_p2v3[hue].values
    _y2 = df_p2v3_mean[hue].values
    
    c = [palette[lbl] for lbl in df_p2v3[clst]]
    
    vmax = np.percentile(_y,95) 
    vmin = np.percentile(_y,5)
    
    g = ax.scatter(_x, _y, c=c, s=3, edgecolor='none')
    g2 = ax.plot(bins_mid, _y2, '-o', color='k', markersize=5)
    
    ax.set_title(col+'_'+var_i2n[col])
    ax.set_ylim([vmin, vmax])
    ax.grid(False)
    sns.despine(ax=ax)
plt.show()

In [ ]:
x = 'ptime'
_x = df_p2v3[x].values
df_p2v3['bin_nrm'], bins = pd.qcut(_x, 20, retbins=True)
df_p2v3_mean = df_p2v3.filter(regex="nrm$", axis=1).groupby('bin_nrm').mean(numeric_only=True)

bins_mid = np.mean(np.vstack([bins[:-1], bins[1:]]), axis=0)

clst = 'clst_l2'
palette = palettes[clst]

fig, axs = plt.subplots(4,5,figsize=(5*4,4*4), sharex=True) #, sharey=True)
for col, ax in zip(var_order_manual.index, axs.flat):
    hue = col+'_nrm'
    _y = df_p2v3[hue].values
    _y2 = df_p2v3_mean[hue].values
    
    vmax = np.percentile(_y,95) 
    vmin = np.percentile(_y,5)
    
    c = [palette[lbl] for lbl in df_p2v3[clst]]
    
    g = ax.scatter(_x, _y, c=c, s=3, edgecolor='none')
    g2 = ax.plot(bins_mid, _y2, '-o', color='k', markersize=5)
    
    ax.set_title(col+'_'+var_i2n[col])
    ax.set_ylim([vmin, vmax])
    ax.grid(False)
    sns.despine(ax=ax)
    # ax.set_xlim(xmin=0, xmax=400)
    # ax.set_xticks([0,100,200,300,400])
# ax.invert_yaxis()
plt.show()

In [ ]:
row_order = np.argsort(df_p2v3['ptime'].values)
col_order = basicu.get_index_from_array(mat_ftrs.columns, var_order_manual.index.values[:-1])

fig = plt.figure(figsize=(20,8))
axd = fig.subplot_mosaic("A\n"*12+"B\nC\nD\nE\nF")
ax = axd["A"]
col_idx = mat_ftrs.columns[col_order].values.astype(str)
col_names = var_i2n[col_idx].values.astype(str)
yticklabels = np.char.add(np.char.add(col_names, ' '), col_idx)

sns.heatmap(
    mat_ftrs.iloc[row_order,col_order].T,
    xticklabels=False,
    yticklabels=yticklabels,
    cmap='coolwarm',
    cbar_kws=dict(shrink=0.5, label='zscore log2(1+norm expr.)'),
    vmax=3, vmin=-3,
    ax=ax,
)
ax.set_ylabel('Cells')
# ax.set_xticklabels()

for i, (ax, col) in enumerate(zip(
    (axd["B"], axd["C"], axd["D"]),
    ('clst_l1', 'clst_l2', 'clst_l3'),
    )):
    # vec = adata3v2.obs[col][row_order].values
    sns.heatmap(
        adata3v3.obs[[col]].rename(columns={col: col.replace('clst_l', 'L')}).iloc[row_order].T,
        xticklabels=False,
        cmap='jet',
        cbar_kws=dict(aspect=5, label='', ticks=[]),
        ax=ax,
    )
    ax.set_ylabel('')
    # positions = np.cumsum(np.unique(vec, return_counts=True)[1])
    # ax.hlines(positions, xmin=-0.2, xmax=1, linewidth=1, linestyle='-', color='k', clip_on=False)
    # for i, pos in enumerate(positions):
    #     ax.text(0, pos, i+1, color='white', fontsize=15)

ax = axd["E"]
g = sns.heatmap(
    mat_prjs.iloc[row_order].T,
    xticklabels=False,
    cmap='Greys',
    cbar_kws=dict(aspect=5, ),
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points

ax = axd["F"]
g = sns.heatmap(
    adata3v2.obs[['y']].iloc[row_order].T, 
    xticklabels=False,
    cmap='Greys',
    cbar_kws=dict(aspect=5, label='um'),
    # vmin=0,
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points